## Overview of Assignment 4

This assignment focuses on exploring and implementing advanced concepts and techniques in information retrieval. The primary objectives are to build Retrieval Augumentation Generation, and learn about Language Models

## Enter your details below

## Name

## Banner ID

## GitHub Link of your Assingment 4

## Q1 : Setting up the libraries and the environment

In [4]:
import sys
print(sys.executable)

c:\Users\AVuser\AppData\Local\Programs\Python\Python312\python.exe


In [6]:
!{sys.executable} -m pip install -q --upgrade pip

!{sys.executable} -m pip install -q langchain faiss-cpu sentence-transformers transformers datasets tiktoken openai chromadb langchain-community

'{sys.executable}' is not recognized as an internal or external command,
operable program or batch file.
'{sys.executable}' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
import langchain
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate

from transformers import AutoTokenizer, AutoModel, T5EncoderModel
from sentence_transformers import SentenceTransformer
import faiss
from datasets import load_dataset

import os
import numpy as np
import torch

## Q2:  Data Preprocessing and Model Selection

In [13]:
ROOT_DIR = "pandas_code"
MODEL_NAME = "Salesforce/codet5p-110m"
MAX_TOKENS = 512
BATCH_SIZE = 8
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [14]:
# 1. Load all .py code files
def load_code_files(root_dir):
    code_texts = []
    for subdir, _, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.py'):
                filepath = os.path.join(subdir, file)
                with open(filepath, 'r', encoding='utf-8') as f:
                    code_texts.append(f.read())
    return code_texts

code_chunks = load_code_files(ROOT_DIR)
print(f"Loaded {len(code_chunks)} Python files from '{ROOT_DIR}'.")

Loaded 1506 Python files from 'pandas_code'.


In [15]:
# 2. Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = T5EncoderModel.from_pretrained(MODEL_NAME).to(DEVICE)
model.eval()

OSError: Salesforce/codet5p-110m is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

In [ ]:
# 3. Token chunking
def chunk_tokens(tokens, max_len=MAX_TOKENS):
    return [tokens[i:i+max_len] for i in range(0, len(tokens), max_len)]

def tokens_to_text(tokens):
    return tokenizer.convert_tokens_to_string(tokens)

In [ ]:
# 4. Prepare code chunks (safe version with truncation)
chunked_texts = []
for code in code_chunks:
    encoded = tokenizer(code, return_tensors="pt", truncation=True, max_length=MAX_TOKENS)
    truncated_text = tokenizer.decode(encoded['input_ids'][0], skip_special_tokens=True)
    chunked_texts.append(truncated_text)

Token indices sequence length is longer than the specified maximum sequence length for this model (515 > 512). Running this sequence through the model will result in indexing errors


Prepared 13683 text chunks for embedding.


: 

In [ ]:
# 5. Generate embeddings
def get_embeddings(texts, tokenizer, model, device=DEVICE, batch_size=BATCH_SIZE):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        encoded = tokenizer(batch, padding=True, truncation=True, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**encoded)
            hidden_states = outputs.last_hidden_state  # [B, T, H]
            mask = encoded['attention_mask'].unsqueeze(-1).expand(hidden_states.size())
            summed = torch.sum(hidden_states * mask, dim=1)
            counts = torch.clamp(mask.sum(1), min=1e-9)
            mean_pooled = summed / counts
            embeddings.append(mean_pooled.cpu().numpy())
    return np.vstack(embeddings)

print("Computing embeddings...")
embeddings = get_embeddings(chunked_texts, tokenizer, model)
print(f"Generated embeddings of shape: {embeddings.shape}")

Computing embeddings...


In [ ]:
# 6. Save embeddings using FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

faiss.write_index(index, "code_index.faiss")
print("Saved FAISS index to 'code_index.faiss'")

## Q3: Implementing RAG using LangChain for different queries

## Q4 : Modify and evaluate the different components of RAG

## Q5: Selecting and implementing a pretrained model for a new task